# 用户数据处理
（只取训练集和测试集中出现的用户ID）

数据来源于Kaggle竞赛：Event Recommendation Engine Challenge，根据
events they’ve responded to in the past
user demographic information
what events they’ve seen and clicked on in our app
用户对某个活动是否感兴趣

竞赛官网：
https://www.kaggle.com/c/event-recommendation-engine-challenge/data

用户描述信息在users.csv文件：共7维特征
user_id
locale：地区，语言
birthyear：出身年
gender：性别
joinedAt：用户加入APP的时间，ISO-8601 UTC time
location：地点
timezone：时区

# 导入工具包

In [1]:
import pandas as pd

import numpy as np
import scipy.sparse as ss
import scipy.io as sio

#保存数据
import pickle

#event的特征需要编码
from utils import FeatureEng
from sklearn.preprocessing import normalize
#相似度/距离
import scipy.spatial.distance as ssd

总的用户数目超过训练集和测试集中的用户，
为节省处理时间和内存，先去处理train和test，得到竞赛需要用到的事件和用户
然后对在训练集和测试集中出现过的事件和用户建立新的ID索引
先运行user_event.ipynb,
得到事件列表文件：PE_userIndex.pkl

# 读取之前算好的测试集和训练集中出现过的用户

In [2]:
#读取训练集和测试集中出现过的用户列表
userIndex = pickle.load(open("PE_userIndex.pkl", 'rb'))
n_users = len(userIndex)

print("number of users in train & test :%d" % n_users)

number of users in train & test :3391


# 处理users.csv --> 特征编码、用户之间的相似度

In [3]:
#读取数据
users = pd.read_csv("users.csv")
users.head()

,user_id,locale,birthyear,gender,joinedAt,location,timezone
0,3197468391,id_ID,1993,male,2012-10-02T06:40:55.524Z,Medan Indonesia,480.0
1,3537982273,id_ID,1992,male,2012-09-29T18:03:12.111Z,Medan Indonesia,420.0
2,823183725,en_US,1975,male,2012-10-06T03:14:07.149Z,Stratford Ontario,-240.0
3,1872223848,en_US,1991,female,2012-11-04T08:59:43.783Z,Tehran Iran,210.0
4,3429017717,id_ID,1995,female,2012-09-10T16:06:53.132Z,NaN,420.0


In [4]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38209 entries, 0 to 38208
Data columns (total 7 columns):
user_id      38209 non-null int64
locale       38209 non-null object
birthyear    38209 non-null object
gender       38100 non-null object
joinedAt     38152 non-null object
location     32745 non-null object
timezone     37773 non-null float64
dtypes: float64(1), int64(1), object(5)
memory usage: 2.0+ MB


In [6]:
FE = FeatureEng()

#locale	birthyear	gender	joinedAt	location	timezone
#去掉user_id列
n_cols = users.shape[1] - 1
cols = ['LocaleId', 'BirthYearInt', 'GenderId', 'JoinedYearMonth', 'CountryId', 'TimezoneInt']

#users编码后的特征
#userMatrix = np.zeros((n_users, n_cols), dtype=np.int)
userMatrix = ss.dok_matrix((n_users, n_cols))

for u in range(users.shape[0]): 
    userId = str(users.loc[u,'user_id'])
    
    #if userIndex.has_key(userId):  #在训练集或测试集中出现
    if userId in userIndex:
        i = userIndex[userId]
    
        userMatrix[i, 0] = FE.getLocaleId(users.loc[u,'locale'])
        userMatrix[i, 1] = FE.getBirthYearInt(users.loc[u,'birthyear'])
        userMatrix[i, 2] = FE.getGenderId(users.loc[u,'gender'])
        userMatrix[i, 3] = FE.getJoinedYearMonth(users.loc[u,'joinedAt'])
        
        #由于地点的写法不规范，该编码似乎不起作用（所有样本的特征都被编码成0了）
        userMatrix[i, 4] = FE.getCountryId(users.loc[u,'location'])
        
        userMatrix[i, 5] = FE.getTimezoneInt(users.loc[u,'timezone'])

# 归一化用户矩阵
userMatrix = normalize(userMatrix, norm="l2", axis=0, copy=False)
sio.mmwrite("US_userMatrix", userMatrix)


# 计算用户相似度矩阵，之后用户推荐系统
userSimMatrix = ss.dok_matrix((n_users, n_users))

#读取在测试集和训练集中出现的用户对
uniqueUserPairs = pickle.load(open("FE_uniqueUserPairs.pkl", 'rb'))

#对角线元素
for i in range(0, n_users):
    userSimMatrix[i, i] = 1.0
    
#对称
for u1, u2 in uniqueUserPairs:
    #i = userIndex[u1]
    #j = userIndex[u2]
    i = u1
    j = u2
    if not (i, j) in userSimMatrix:
        #Person相关系数做为相似度度量
        #特征：国家（locale、location）、年龄、性别、时区、地点
        #usim = ssd.correlation(userMatrix[i,:],
            #userMatrix[j,:])
    
        usim = ssd.correlation(userMatrix.getrow(i).todense(),
          userMatrix.getrow(j).todense())
        userSimMatrix[i, j] = usim
        userSimMatrix[j, i] = usim
    
sio.mmwrite("US_userSimMatrix", userSimMatrix)

In [7]:
print(userId)

125781176


In [9]:
print(userMatrix)
print(type(userMatrix))

  (230, 0)	0.021628856085564065
  (3139, 0)	0.011957416372506963
  (123, 0)	0.021628856085564065
  (1599, 0)	0.014243393031956823
  (2739, 0)	0.021628856085564065
  (816, 0)	0.011957416372506963
  (2351, 0)	0.021628856085564065
  (1016, 0)	0.011957416372506963
  (2944, 0)	0.017144824945873955
  (1364, 0)	0.021628856085564065
  (335, 0)	0.021628856085564065
  (1135, 0)	0.011957416372506963
  (881, 0)	0.021628856085564065
  (1590, 0)	0.005451175110995821
  (542, 0)	0.011166116759620473
  (560, 0)	0.011957416372506963
  (2907, 0)	0.021628856085564065
  (3213, 0)	0.011957416372506963
  (98, 0)	0.021628856085564065
  (1065, 0)	0.025585354149996516
  (1915, 0)	0.011957416372506963
  (1563, 0)	0.011957416372506963
  (922, 0)	0.021628856085564065
  (852, 0)	0.021628856085564065
  (2290, 0)	0.021628856085564065
  :	:
  (2713, 5)	-0.01303142053748795
  (978, 5)	0.020850272859980718
  (3054, 5)	0.010425136429990359
  (326, 5)	0.014334562591236744
  (1539, 5)	0.01824398875248313
  (2884, 5)	0.0104

In [10]:
print(u1,u2)

2630 1518


In [11]:
print(userSimMatrix)

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0
  (5, 5)	1.0
  (6, 6)	1.0
  (7, 7)	1.0
  (8, 8)	1.0
  (9, 9)	1.0
  (10, 10)	1.0
  (11, 11)	1.0
  (12, 12)	1.0
  (13, 13)	1.0
  (14, 14)	1.0
  (15, 15)	1.0
  (16, 16)	1.0
  (17, 17)	1.0
  (18, 18)	1.0
  (19, 19)	1.0
  (20, 20)	1.0
  (21, 21)	1.0
  (22, 22)	1.0
  (23, 23)	1.0
  (24, 24)	1.0
  :	:
  (2708, 34)	0.7551414085441245
  (2654, 317)	0.6285572330561726
  (317, 2654)	0.6285572330561726
  (3258, 988)	7.902592221387117e-06
  (988, 3258)	7.902592221387117e-06
  (2583, 2071)	0.754535687381116
  (2071, 2583)	0.754535687381116
  (2656, 1958)	0.3624753727294502
  (1958, 2656)	0.3624753727294502
  (896, 75)	0.17879821780629457
  (75, 896)	0.17879821780629457
  (1567, 2477)	1.3678021035152539
  (2477, 1567)	1.3678021035152539
  (2661, 1231)	0.7339724865689136
  (1231, 2661)	0.7339724865689136
  (2382, 992)	0.7615419131985823
  (992, 2382)	0.7615419131985823
  (2347, 815)	5.054317115726903e-06
  (815, 2347)	5.054317115726903e-